In [58]:
import config
import torch
from generator import Generator
from discriminator import Discriminator
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from utils import save_checkpoint, load_checkpoint, save_some_examples




disc = Discriminator(in_channels=3).to(config.DEVICE)
gen = Generator(in_channels=3, features=64).to(config.DEVICE)

opt_disc = optim.Adam(
    disc.parameters(),
    lr=config.LEARNING_RATE,
    betas=(0.5, 0.999),
)
opt_gen = optim.Adam(
    gen.parameters(),
    lr=config.LEARNING_RATE,
    betas=(0.5, 0.999),
)

BCE = nn.BCEWithLogitsLoss()
L1_LOSS = nn.L1Loss()

load_checkpoint(
    "./checkpoints/gen_0.pth.tar",
    gen,
    opt_gen,
    config.LEARNING_RATE,
)
load_checkpoint(
    "./checkpoints/disc_0.pth.tar",
    disc,
    opt_disc,
    config.LEARNING_RATE,
)

=> Loading checkpoint
=> Loading checkpoint


In [60]:

depth_path = './celeba/d_test/000001_depth.png'
face_path = './celeba/a_test/000001.jpg'
depth = Image.open(depth_path).convert('RGB')
face = Image.open(face_path).convert('RGB')

transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),  # Resize the images to a smaller size
        transforms.ToTensor(),
        # transforms.RandomHorizontalFlip(),  # Flip the images horizontally
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize the images
    ]
)


depth = transform(depth)
depth = torch.unsqueeze(depth, 0)

face = transform(face)
face =  torch.unsqueeze(face, 0)

from torchvision.utils import save_image

def save_some_examples(gen, x, y, epoch=0, folder='./evaluation/manual_saves'):

    x, y = x.to(config.DEVICE), y.to(config.DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()

save_some_examples(gen, depth, face, 0)